In [ ]:
import os
import numpy as np
import azureml.core
import shutil
from azureml.core import Workspace, Experiment, Dataset
from sklearn.datasets import load_diabetes
from azureml.core.compute import ComputeTarget

print("# [SDK Check]")
print("SDK version:", azureml.core.VERSION)
print("*"*60)

print("# [WS Check]")
ws = Workspace.from_config()
ws_key = ['WS Name', 'RG', 'Region', 'Subscription']
ws_val = [ws.name, ws.resource_group, ws.location, ws.subscription_id]
for k,v in zip(ws_key, ws_val):
    print(f'{k}: {v}')
print("*"*60)

experiment_name = 'train-on-remote-vm2'
exp = Experiment(workspace=ws, name=experiment_name)
script_folder = './vm-run'
os.makedirs(script_folder, exist_ok=True)

print("# [Get DS]")
# Get the default datastore
ds = ws.get_default_datastore()
ds_key = ['DS Name', 'DS Type', 'Storage Account Name', 'BLOB Contaner Name']
ds_val = [ds.name, ds.datastore_type, ds.account_name, ds.container_name]
for k,v in zip(ds_key, ds_val):
    print(f'{k}: {v}')
print("*"*60)

print("# [Upload to DS]")
training_data = load_diabetes()
np.save(file='./features.npy', arr=training_data['data'])
np.save(file='./labels.npy', arr=training_data['target'])
ds.upload_files(['./features.npy', './labels.npy'], target_path='diabetes', overwrite=True)
ds_paths = [(ds, 'diabetes/')]
dataset = Dataset.File.from_files(path = ds_paths)
shutil.copy('./train.py', os.path.join(script_folder, 'train.py'))
print("*"*60)

print("# [Remote VM Check]")
global attached_dsvm_compute
try:
    compute_target_name = 'dsvm'  # Your Remote VM Resource Name
    attached_dsvm_compute = ComputeTarget(workspace=ws, name=compute_target_name)
    print(f'Found Remote VM Resoure Name: {compute_target_name}')
except Exception as e:
    print(f'Error: {e}')

In [ ]:
from uuid import uuid4
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ScriptRunConfig

# 1: Remote VM 先の Python 環境使用ケース
conda_env = Environment("conda-env2")
# NOTE: Remote VM (DSVM-ubuntu 18.04 LTS Gen2) 先の Python 環境を使用するフラグ
conda_env.python.user_managed_dependencies = True
############################################

# 2: カスタム Python 環境使用ケース
sklearn_env = Environment.from_conda_specification(
    name = 'sklearn-env', 
    file_path = './conda_dependencies.yml'
)
############################################

script_arguments = [
    '--data-folder', 
    dataset.as_named_input('diabetes').as_mount(f'/tmp/{uuid4()}')
]

src = ScriptRunConfig(
    source_directory=script_folder, 
    script='train.py', 
    arguments=script_arguments,  # pass the dataset as a parameter to the training script
    compute_target = attached_dsvm_compute,
    environment = sklearn_env
    # environment = conda_env
)


In [ ]:
from azureml.widgets import RunDetails

run = exp.submit(config=src)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

## Clean up compute resource

Use ```detach()``` to detach an existing DSVM from Workspace without deleting it. Use ```delete()``` if you created a new ```DsvmCompute``` and want to delete it.

In [ ]:
# dsvm_compute.detach()
# dsvm_compute.delete()